# Notebook for testing remote providers
This notebook is a template for the testing steps of writing a remote provider to wrap an external API

## Part 1: Getting a feel for the remote API
This section walks through sending and receiving ordinary python requests to the AirNow API. You can adapt this section to whatever API you are working on. The key point is to understand what parameters the API accepts, and what it sends back in response.

In [1]:
import requests

In [2]:
aqi_url = "https://airnowapi.org/aq/data/"
api_key = "463BB454-9613-4D69-A6D0-4B57320FE25A"

In [3]:
options = {
        "url": "https://airnowapi.org/aq/data/",
        "start_date": "2014-09-23",
        "start_hour_utc": "21",
        "end_date": "2014-09-23",
        "end_hour_utc": "23",
        "parameters": "o3,pm25",
        "bbox": ["-90.806632,24.634217,-71.119132,45.910790"],
        "data_type": "a",
        "format": "application/json",
        "api_key": "463BB454-9613-4D69-A6D0-4B57320FE25A"
    }


In [4]:

request_params = {
    #"startdate": options["start_date"] + "T" + options["start_hour_utc"],
    #"enddate": options["end_date"] + "T" + options["end_hour_utc"],
    "parameters": options["parameters"],
    "bbox": options["bbox"],
    "datatype": options["data_type"],
    "format": options["format"],
    "api_key": options["api_key"],
    "page":2,
    "page_size":200,
 }

try:
    # Request AirNowAPI data
    print("Requesting AirNowAPI data...")
    
    # Perform the AirNow API data request
    response = requests.get(aqi_url, params=request_params)
    response.raise_for_status()
except:
    print("error")



Requesting AirNowAPI data...


### Examine the response

In [ ]:
response.json()

This part is not necessary, but it may be easier to visualize the results this way

In [19]:
features=[]
for res in response.json():
    feat = geodesic.Feature(geometry= {"type": "Point", "coordinates": [res['Latitude'], res['Longitude']]},
                            properties=res,
                            datetime= datetime.strptime(res['UTC'], "%Y-%m-%dT%H:%M"))
    features.append(feat)


                                       

In [20]:
fc = geodesic.FeatureCollection(features=features)

In [21]:
fc.gdf

,Latitude,Longitude,UTC,Parameter,Unit,AQI,Category,datetime,geometry
0,29.763900,-90.765300,2024-06-07T13:00,PM2.5,UG/M3,59,2,2024-06-07T13:00:00+00:00,POINT (29.76390 -90.76530)
1,29.763900,-90.765300,2024-06-07T13:00,OZONE,PPB,13,1,2024-06-07T13:00:00+00:00,POINT (29.76390 -90.76530)
2,33.750833,-90.734167,2024-06-07T13:00,OZONE,PPB,30,1,2024-06-07T13:00:00+00:00,POINT (33.75083 -90.73417)
3,33.750833,-90.734167,2024-06-07T13:00,PM2.5,UG/M3,19,1,2024-06-07T13:00:00+00:00,POINT (33.75083 -90.73417)
4,38.490200,-90.705200,2024-06-07T13:00,OZONE,PPB,17,1,2024-06-07T13:00:00+00:00,POINT (38.49020 -90.70520)
...,...,...,...,...,...,...,...,...,...
1136,45.373300,-71.251400,2024-06-07T13:00,OZONE,PPB,21,1,2024-06-07T13:00:00+00:00,POINT (45.37330 -71.25140)
1137,45.373300,-71.251400,2024-06-07T13:00,PM2.5,UG/M3,7,1,2024-06-07T13:00:00+00:00,POINT (45.37330 -71.25140)
1138,44.308132,-71.217639,2024-06-07T13:00,OZONE,PPB,23,1,2024-06-07T13:00:00+00:00,POINT (44.30813 -71.21764)
1139,41.683300,-71.169701,2024-06-07T13:00,OZONE,PPB,28,1,2024-06-07T13:00:00+00:00,POINT (41.68330 -71.16970)


## Part 2: Testing the deployed remote provider as a Boson Dataset

In [6]:
import geodesic
from geodesic.cql import CQLFilter
from datetime import datetime as _datetime
from geodesic import mapping

In [179]:
proj_name = 'remote-provider-test'
proj = geodesic.Project(name=proj_name, alias="Testing proj for remote provider")
proj.create()

In [180]:
geodesic.set_active_project(proj)

{'name': 'air-quality-test',
 'alias': 'Testing proj for air quality datasets',
 'uid': '3391b7537426f6344f75f009878bd264a749c5b1',
 'owner': 'auth0|621d9542ecd81c0069eee8ed'}

In [4]:
url = "https://air-now-remote-provider-azwzjbkrwq-uc.a.run.app"
ds = geodesic.Dataset.from_remote_provider('airnow', url)

In [5]:
ds.save()

dataset:*:*:*:airnow

In [153]:
dt = [_datetime(2024, 1, 20), _datetime(2024, 1, 20, 5)]

In [154]:
dt

[datetime.datetime(2024, 1, 20, 0, 0), datetime.datetime(2024, 1, 20, 5, 0)]

In [156]:
colorado_bbox = [-109.0, 36.9, -102.0, 41.0]

In [287]:
test = ds.search(bbox=colorado_bbox) #, datetime=dt)

In [134]:
test['features'][0]['properties']['AQI']

15

In [204]:
ds_view = ds.view('colorado-aqi', bbox=colorado_bbox)

In [205]:
ds_view.save()

ObjectWidget(object_value=[{'alias': 'colorado-aqi', 'name': 'colorado-aqi', 'project': '3391b7537426f6344f75f…

In [ ]:
### Display a FeatureCollection on a map

In [211]:
from ipyleaflet import VectorTileLayer



#layer = VectorTileLayer(name="pm2.5_vector", url=share_url)
m = mapping.Map()
m.add_feature_collection("pm2.5", test, on_click=True)
#m.add_layer(layer)

m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
### Display a vector tile service on a map

In [206]:
token = ds_view.share_as_ogc_tiles_service(600)
share_url = token.get_ogc_vector_tile_url()

In [202]:
share_url

'https://api.geodesic.seerai.space/ted/api/v1/share/f4723f7c842084d414a7d8065e179402ed13c84adf1f08d36062b05790bdfe0d/collections/airnow/tiles/WebMercatorQuad/{z}/{y}/{x}.mvt'

In [ ]:
### Share to ESRI geoservices

In [200]:
arc_token = ds.share_as_arcgis_service(600)
arcgis_url = arc_token.get_vector_tile_service_url()

In [201]:
arcgis_url

'https://api.geodesic.seerai.space/ted/api/v1/share/315154b4351e30dc0f488d7f5e5c6784f9f3ff6d74af86f843e2c6e935f799e0/rest/services/airnow/VectorTileServer'